# Intro

Server-Sent Events (SSE) are a web technology that allow servers to push real-time updates to web clients over a single HTTP connection. Unlike traditional HTTP requests where the client must repeatedly ask for new data, SSE creates a one-way channel from the server to the client that stays open, allowing the server to send updates whenever they're available.

![SSE Diagram](static_blog_imgs/sse_diag.png)

Some key characteristics of SSE:

- One-directional: Data flows only from server to client
- Uses standard HTTP (unlike WebSockets which use a different protocol)
    - WebSockets are bidirectional, meaning data can flow both ways - from server to client and from client to server over the same connection. This makes WebSockets suitable for applications requiring real-time communication in both directions, like chat applications or multiplayer games.
    - In contrast, SSE are strictly one-directional (server to client only).
    - This fundamental difference in communication flow is one of the key factors to consider when choosing between these technologies for your application.
- Automatically reconnects if the connection is lost
- Lightweight compared to alternatives like WebSockets
- Built into browsers with the EventSource API

# Example 1: Basic SSE Implementation

Let's look at a basic example first to see how SSE works.
In this example an SSE connection is setup when the page is loaded.

In [1]:
# | echo: false
from IPython.display import Markdown, display


def import_python_as_markdown(file_path):
    with open(file_path, "r") as file:
        content = file.read()
    return f"```python\n{content}\n```"


file_path = "sse1.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
# ruff: noqa: F403, F405
from asyncio import sleep

from fasthtml.common import *
from monsterui.all import *

app, rt = fast_app(
    hdrs=(
        Theme.blue.headers(highlightjs=True),  # monsterui styling
        Script(src="https://unpkg.com/htmx-ext-sse@2.2.1/sse.js"),  # Include this to use the SSE extension
    ),
    live=True,
)


@rt("/")
def index():
    return Container(
        H3("Intro SSE Example"),
        Div(
            P("The contents of this <div> will be updated in real time with each SSE message received."),
            hx_ext="sse",  # To connect to an SSE server, use the hx_ext="sse" attribute to install the extension on that HTML element
            sse_swap="EventName",  # This default event name is "message" if we don't specify it otherwise
            sse_connect="/sse-stream",  # This is the URL of the SSE endpoint we create and connect to
            hx_swap="beforeend show:bottom",  # Determines how the content will be inserted into that target element. Here, each new message is added at the end of the div and the page automatically scrolls to show the new message
            hx_target=None,  # None is the default. By not specifying a target for the swap, it defaults to the element that triggered the request i.e. id="sse-content"
            id="sse-content",
        ),
    )


async def message_generator():
    # This sse_message function converts an HTML element into the specific format required for Server-Sent Events (SSE) streaming.
    # The first argument is an FT component (FastHTML element) that you want to send via SSE.
    # The second argument is the name of the SSE event (defaults to "message" if not specified).
    # It must match the sse_swap attribute above i.e. event="EventName"
    
    for i in range(10):
        yield sse_message(Div(P(f"message number {i}")), event="EventName")  
        await sleep(0.5)

    yield sse_message(Div(P("DONE")), event="EventName") 


@rt("/sse-stream")
async def sse_stream():
    return EventStream(message_generator())


serve(port=5010)

```

** TODO: Run The Example as A Video Or Embedded App In the BLOG!**

## Breaking Down The Example in Details

### Initial Page Load

When a user navigates to the root URL, the browser sends a `GET` request to the server.
The server responds with the HTML generated by the `index()` function, which creates:

- A container with a heading "Intro SSE Example"

- A div with ID "sse-content" containing an introductory paragraph. This div has several HTMX attributes that configure SSE behavior


- The browser renders this initial HTML, showing the heading and the introductory paragraph.

### SSE Connection Establishment

The browser sees the `hx_ext="sse"` and `sse_connect="/sse-stream"` attributes on the div and recognizes that it needs to establish a Server-Sent Events connection.
The browser automatically opens an `EventSource` connection to the `/sse-stream` endpoint.
On the server, when this connection request arrives, it triggers the `sse_stream()` function, which:

- Creates a new instance of the `message_generator()` coroutine
- Wraps it in an `EventStream` response object
- Sends the appropriate HTTP headers to establish an SSE connection


### Message Streaming Process

Once the connection is established, the `message_generator()` coroutine begins execution:
For each iteration (0-9):

- It creates an HTML message containing "message number {i}"
- Converts this to SSE format with the event name "EventName"
- Yields this message, which is immediately sent to the browser
- Pauses for 0.5 seconds using `await sleep(0.5)`
- During this pause, the server can handle other requests because of the use of `async`


After the 10 numbered messages, it sends a final message containing "DONE".

### Client-Side Processing

As each SSE message arrives at the browser, HTMX intercepts it because of the `hx_ext="sse"` attribute.
It checks the event name in the message ("EventName") and matches it against the `sse_swap="EventName"` attribute.
Since they match, HTMX processes this message


The content of each message is inserted into the div according to the `hx_swap="beforeend show:bottom"` attribute:

- `beforeend`: Each new message is added at the end of the existing content
- `show:bottom`: The page automatically scrolls to show the new content

The user sees each message appear approximately every half second, with the page scrolling to keep the latest message visible.

### Connection Behavior After Completion

After the final "DONE" message, the generator is exhausted, but the SSE connection doesn't automatically close.
The browser's `EventSource` implementation will detect the end of the stream and automatically attempt to reconnect after a brief delay.
This reconnection will trigger another call to `sse_stream()`, creating a new instance of `message_generator()`, and the sequence will start over.
This cycle will continue indefinitely, with the div accumulating more and more messages, unless the page is navigated away from or the connection is explicitly closed.

This was a surprise to me when first learning about SSE. I expected the connection to close after the final message.



# Example 2: Triggering SSE from a Button Click

In this example, we'll see how to trigger SSE from a button click event instead of the initial page load.





# Server Sent Events in FastHTML

The [htmx.org website](https://htmx.org/extensions/sse/) has a great explanation of how to use the htmx Server Sent Event (SSE) extension. Here's a quick summary. We will use the `FastHTML` naming convention for the attributes. For example,
`hx-ext` becomes `hx_ext`, `sse-connect` becomes `sse_connect`, etc.

- `hx_ext="sse"` - This is required to use the SSE extension.
- `sse_connect="<url>"` - The URL of the SSE endpoint
- SSE works just like any HTTP request. The client first initiates a connection, which can be triggered 
by things such as page loads, button clicks, etc. On the initial request which establishes the connection, the client can also send parameters along with the request. Once the connection is established, the server can send messages to the client at any time until the connection is closed.
- `sse_swap="<message-name>"` - The name of the message to swap into the DOM.
- `hx_trigger="sse:<message-name>"` - SSE messages can also trigger HTTP callbacks using the hx-trigger attribute.
- `sse_close=<message-name>` - To close the EventStream gracefully when that message is received. This might be helpful if you want to send information to a client that will eventually stop.